In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors
from matplotlib.colors import LogNorm
import seaborn as sns
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import ode
import scipy.ndimage
import glob
import os
from fractions import Fraction

In [7]:
is_ode = "yes"
is_contour = "yes"

os.makedirs("./virus_change_csv_2022/", exist_ok=True)
os.makedirs("./H_change_csv_2022/", exist_ok=True)
os.makedirs("./M_change_csv_2022/", exist_ok=True)


# os.makedirs("./v_int_change_csv_2022/", exist_ok=True)
# os.makedirs("./v_int_contour_csv_2022/", exist_ok=True)

os.makedirs("./virus_change_pdf_2022/", exist_ok=True)
os.makedirs("./H_change_pdf_2022/", exist_ok=True)
os.makedirs("./M_change_pdf_2022/", exist_ok=True)
# os.makedirs("./v_int_change_pdf_2022/", exist_ok=True)
# os.makedirs("./v_int_contour_pdf_2022/", exist_ok=True)


In [8]:
# colormapをカスタマイズする
from matplotlib.colors import LinearSegmentedColormap

def generate_cmap(colors):
    """自分で定義したカラーマップを返す"""
    values = range(len(colors))

    vmax = np.ceil(np.max(values))
    color_list = []
    for v, c in zip(values, colors):
        color_list.append( ( v/ vmax, c) )
    return LinearSegmentedColormap.from_list('custom_cmap', color_list)

##### cmap for c_alpha_beta #####
cm = generate_cmap(['gray', 'black'])

def func_1(x, t, pars):
	#print(v_int)
	#print(solve.t+dt-solve.t)
	#print(solve.integrate(solve.t+dt)[0])
	return [(pars["r"]/(1+pars["d"]))*x[0] - pars["b"]*x[1]*x[0], pars["a"]*x[0] + pars["q"]*x[2] - pars["c"]*x[1], pars["u"]*x[0]]


In [9]:
v_int=0
dt = 0.1
t_end=30

pars1={"r":1,"d":0,"b":1,"a":1,"q":0.1,"c":0.5,"u":1}
pars2={"r":1,"d":0.5,"b":1,"a":1,"q":0.1,"c":0.5,"u":1}

t1_list = np.arange(0.2, 10, 0.2)
t3_list = np.arange(0.2, 15, 0.2)

range_t1 =len(t1_list)
# t2_list = [k for k in t2_list if k >=2]
print(range_t1)
# print(t2_list)
# start=t1_list[2]
# print(start)

49


In [10]:
if is_ode == "yes":
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 14
    plt.rcParams["mathtext.fontset"] = 'stix'
    
    df_v_int_f=pd.DataFrame()
    
    for i in range(range_t1):
        start=t1_list[i]
        t2_list = [k for k in t3_list if  k>=start]
        range_t2=len(t2_list)
        print(range_t2)
#         list_v_int_f=[]
        for j in range(range_t2):
            end=t2_list[j]
            df_v_int=pd.DataFrame()
            df_V=pd.DataFrame()
            df_H=pd.DataFrame()
            df_M=pd.DataFrame()
            x= [0.1,0,0]
            v_int=0
            list_t=[]
            list_v_int=[]
            list_V=[]
            list_H=[]
            list_M=[]
            listname="t1="+str(start)
            ###  array  ####
#             t = np.arange(0,t_steps*dt,dt)
            t0=np.arange(0,start,dt)
            y0 = odeint(func_1, (x[0],x[1],x[2]), t0, args= (pars1,))
            
            ###  list  ###
            t_list=list(t0)
            V_list=list(y0[:,0])
            H_list=list(y0[:,1])
            M_list=list(y0[:,2])
            
            list_t = t_list
            list_V= V_list
            list_H= H_list
            list_M= M_list
            
            ###  array  ####
            x=[V_list[-1], H_list[-1], M_list[-1]]
            t1=np.arange(start-dt,end,dt)
            y1 = odeint(func_1, (x[0],x[1],x[2]), t1, args= (pars2,))
            ###  list  ###
            t_list=list(t1)
            V_list=list(y1[:,0])
            H_list=list(y1[:,1])
            M_list=list(y1[:,2])
            
            list_t.extend(t_list[1:-1])
            list_V.extend(V_list[1:-1])
            list_H.extend(H_list[1:-1])
            list_M.extend(M_list[1:-1])
            
            ###  array  ####
            x=[V_list[-1], H_list[-1], M_list[-1]]
            t2=np.arange(end-dt,t_end,dt)
            y2 = odeint(func_1, (x[0],x[1],x[2]), t2, args= (pars1,))
            
            ###  list  ###
            t_list=list(t2)
            V_list=list(y2[:,0])
            H_list=list(y2[:,1])
            M_list=list(y2[:,2])
            
            list_t.extend(t_list[1:-1])
            list_V.extend(V_list[1:-1])
            list_H.extend(H_list[1:-1])
            list_M.extend(M_list[1:-1])

            df_V[listname]=list_V
            df_H[listname]=list_H
            df_M[listname]=list_M
#             df_v_int[listname]=list_v_int
            df_V.index=list_t
            df_H.index=list_t
            df_M.index=list_t
#             df_v_int.index=list_t
#             list_v_int_f.append(v_int)

            ##### virus
            pars_txt=["t1="+str(start),"t2="+str(end)]
            csvname='./virus_change_csv_2022/virus_change'+"t1="+str(start)+"_t2="+str(end)+'.csv'
            df_V.to_csv(csvname)
            ######
            ax = df_V.plot(color = "black",legend=False)
            ax.set_position([0.13,0.13,0.8,0.8])
            ax.set_yticks([0,0.2,0.4 ,0.6,0.8])
            figname='./virus_change_pdf_2022/virus_change'+"t1="+str(start)+"_t2="+str(end)+'.pdf'
            plt.xlabel("Time",fontsize=20)
            plt.ylabel(r"$V(t)$ ",fontsize=20)
            pars_title = r"$t1=$" + str(start) +" and " + r"$t2=$" + str(end)
            plt.title(pars_title)
            plt.savefig(figname)
            plt.close()

            ###### virus integrated 
#             pars_txt=["t1="+str(start),"t2="+str(end)]
#             csvname='./v_int_change_csv_2022/v_int_change'+"t1="+str(start)+"_t2="+str(end)+'.csv'
#             df_v_int.to_csv(csvname)
#             ######
#             ax = df_v_int.plot(color = "black",legend=False)
#             ax.set_position([0.13,0.13,0.8,0.8])
#             figname='./v_int_change_pdf_2022/v_int_change'+"t1="+str(start)+"_t2="+str(end)+'.pdf'
#             pars_title = r"$t1=$" + str(start) +" and " + r"$t2=$" + str(end)
#             plt.title(pars_title)
#             plt.xlabel("Time",fontsize=20)
#             plt.ylabel(r"$ψ$ ",fontsize=20)
#             plt.savefig(figname)
#             plt.close()
            #######

            #####　T helper
            pars_txt=["t1="+str(start),"t2="+str(end)]
            csvname='./H_change_csv_2022/H_change'+"t1="+str(start)+"_t2="+str(end)+'.csv'
            df_H.to_csv(csvname)
            ######
            ax = df_H.plot(color = "black",legend=False)
            ax.set_position([0.13,0.13,0.8,0.8])
            figname='./H_change_pdf_2022/H_change'+"t1="+str(start)+"_t2="+str(end)+'.pdf'
            plt.xlabel("Time",fontsize=20)
            plt.ylabel(r"$H(t)$ ",fontsize=20)
            pars_title = r"$t1=$" + str(start) +" and " + r"$t2=$" + str(end)
            plt.title(pars_title)
            plt.savefig(figname)
            plt.close()
            ######

            ##### Memory
            pars_txt=["t1="+str(start),"t2="+str(end)]
            csvname='./M_change_csv_2022/M_change'+"t1="+str(start)+"_t2="+str(end)+'.csv'
            df_M.to_csv(csvname)
            ######
            ax = df_M.plot(color = "black",legend=False)
            ax.set_position([0.13,0.13,0.8,0.8])
            figname='./M_change_pdf_2022/M_change'+"t1="+str(start)+"_t2="+str(end)+'.pdf'
            plt.xlabel("Time",fontsize=20)
            plt.ylabel(r"$M(t)$ ",fontsize=20)
            pars_title = r"$t1=$" + str(start) +" and " + r"$t2=$" + str(end)
            plt.title(pars_title)
            plt.savefig(figname)
            plt.close()
            ######

#     print(list_v_int_f)
#     #listname_v_int = "beta=" + str(pars["beta"])
#     df_v_int_f[str(end)]=list_v_int_f

#     csvname='./v_int_contour_csv_2022/'+'t1_'+str(t1_list[-1])+'_t2_'+str(t2_list[-1])+'.csv'
#     df_v_int_f.index = t1_list
#     df_v_int_f.to_csv(csvname)

74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
